In [29]:
#to install with pip tensorflow i had to change the registry value for longpaths.
import pandas as pd
import matplotlib.pyplot as plt
import re
from sklearn.preprocessing import   LabelEncoder
from sklearn.linear_model import LogisticRegression
import numpy as np

In [30]:
df=pd.read_csv('data/Results_TTM.csv',sep=";",header=0)
df=df.drop(columns=["Step status","Current trend","Torque rate min","Torque rate max","Torque rate trend","CVILOGIX","Identifier6","Identifier7","Identifier8","Identifier9","Identifier10","Second transducer torque deviation","Second transducer angle deviation","Result type","Pulse counter","Angle offset","AO torque rate"])

C:\Users\guill\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (45,46,63) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [31]:
df=df[['Result status','Result number','Time result','Pset ID','Step ID','Error Code', 'Torque min','Torque','Torque max','Angle min','Angle','Angle max','Pset name','VIN','Identifier1','Identifier2','Identifier3','Identifier4','Identifier5']]

In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400000 entries, 0 to 399999
Data columns (total 19 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Result status  400000 non-null  object 
 1   Result number  400000 non-null  int64  
 2   Time result    400000 non-null  object 
 3   Pset ID        399901 non-null  float64
 4   Step ID        399901 non-null  float64
 5   Error Code     22688 non-null   object 
 6   Torque min     399901 non-null  float64
 7   Torque         400000 non-null  float64
 8   Torque max     399901 non-null  float64
 9   Angle min      399901 non-null  float64
 10  Angle          400000 non-null  float64
 11  Angle max      399901 non-null  float64
 12  Pset name      399901 non-null  object 
 13  VIN            172013 non-null  object 
 14  Identifier1    172013 non-null  object 
 15  Identifier2    170608 non-null  object 
 16  Identifier3    168406 non-null  object 
 17  Identifier4    13920 non-null

Rename de columns to avoid spaces.

In [33]:
df=df.rename(columns={"Result status":"Result_status","Result number":"Result_number","Pset ID":"Pset_ID","Step ID":"Step_ID","Torque min":"Torque_min","Torque max":"Torque_max","Angle min":"Angle_min","Angle max":"Angle_max","Pset name":"Pset_name",'Error code':'Error_code','Result_status':'Result_status','Time result':'Time_result'})


Vamos a mirar a ver cuales de los pares son NOK, filtramos la columna de resultado

In [34]:
df_nok=df[df['Result_status'].str.contains("NOK")]
df_nok.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22568 entries, 34 to 399969
Data columns (total 19 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Result_status  22568 non-null  object 
 1   Result_number  22568 non-null  int64  
 2   Time_result    22568 non-null  object 
 3   Pset_ID        22568 non-null  float64
 4   Step_ID        22568 non-null  float64
 5   Error Code     22568 non-null  object 
 6   Torque_min     22568 non-null  float64
 7   Torque         22568 non-null  float64
 8   Torque_max     22568 non-null  float64
 9   Angle_min      22568 non-null  float64
 10  Angle          22568 non-null  float64
 11  Angle_max      22568 non-null  float64
 12  Pset_name      22568 non-null  object 
 13  VIN            9477 non-null   object 
 14  Identifier1    9477 non-null   object 
 15  Identifier2    9289 non-null   object 
 16  Identifier3    9301 non-null   object 
 17  Identifier4    1431 non-null   object 
 18  Iden

In [35]:
df_nok['Pset_name'].head()
df_nok[['Pset_name','Result_status']].groupby(by='Pset_name').count().sort_values(by=['Result_status'], ascending=False)

,Result_status
Pset_name,
Seat frame front_35Nm,1109
Armrests 28Nm,990
Slides 22Nm TX40 ISC,920
Cross angle back - Swivel_22Nm,885
Seat frame assy_35Nm_2x59-2_ISC,877
...,...
Backrest segment 22Nm ISC_A_BAU,1
Poka Yoke Check_6xxx,1
Torque_35Nm,1


Vemos que hay un resultado POKA YOKE, es un resultado NOK forzado que se hace para comprobar la herramientas. hay que quitarlos de la muestra.

In [36]:
df = df[~df["Pset_name"].str.contains("Poka",na=False)]

In [37]:
df.astype({'Time_result': 'datetime64[ns, US/Eastern]'}).dtypes

Result_status                        object
Result_number                         int64
Time_result      datetime64[ns, US/Eastern]
Pset_ID                             float64
Step_ID                             float64
Error Code                           object
Torque_min                          float64
Torque                              float64
Torque_max                          float64
Angle_min                           float64
Angle                               float64
Angle_max                           float64
Pset_name                            object
VIN                                  object
Identifier1                          object
Identifier2                          object
Identifier3                          object
Identifier4                          object
Identifier5                          object
dtype: object

creamos una columna nueva la columna de resultado que contiene los tesxtos "NOK" y "OK" en valores numericos.

In [38]:
df['Resultbin']=df['Result_status']=='OK'

In [39]:
df.head()


,Result_status,Result_number,Time_result,Pset_ID,Step_ID,Error Code,Torque_min,Torque,Torque_max,Angle_min,Angle,Angle_max,Pset_name,VIN,Identifier1,Identifier2,Identifier3,Identifier4,Identifier5,Resultbin
0,OK,613029,2020-12-10 23:59:12.0,10.0,1.0,NaN,18.0,20.126,22.0,3.0,40.213,100.0,Fixation screw_(60895R01)_20Nm_1x64-5_IF,NaN,NaN,NaN,NaN,NaN,NaN,True
1,OK,613028,2020-12-10 23:59:08.0,9.0,1.0,NaN,18.0,20.090,22.0,5.0,40.369,100.0,Fixation screw_(318327B01)_20Nm_1x64-4_I,NaN,NaN,NaN,NaN,NaN,NaN,True
2,OK,613027,2020-12-10 23:58:56.0,3.0,1.0,NaN,20.0,22.219,24.0,5.0,40.389,110.0,Sliders 22Nm_4x65-4_ISC,NaN,NaN,NaN,NaN,NaN,NaN,True
3,OK,613026,2020-12-10 23:58:48.0,3.0,1.0,NaN,20.0,22.138,24.0,5.0,33.650,110.0,Sliders 22Nm_4x65-4_ISC,NaN,NaN,NaN,NaN,NaN,NaN,True
4,OK,613025,2020-12-10 23:58:41.0,3.0,1.0,NaN,20.0,22.073,24.0,5.0,43.830,110.0,Sliders 22Nm_4x65-4_ISC,NaN,NaN,NaN,NaN,NaN,NaN,True


Aun tenemos columnas que no van a aportar nada a nuestro modelo.

In [40]:
dfX=df.drop(columns={'Result_status','Result_number','Time_result','Pset_ID','Identifier4','Identifier5','Identifier2','Error Code','Torque','Angle'})

con df.info() vamos a ver cuantas columnas contienen resultados nulos y vamos a hacer que toda la muestra tenga columnas con datos que podamos utilizar.
Con esta accion podemos hacer que nuestro modelo posterior no sea tan bueno prediciendo los resultados de las columnas que tienen nulls.

In [41]:
dfX.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 398943 entries, 0 to 399999
Data columns (total 10 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   Step_ID      398844 non-null  float64
 1   Torque_min   398844 non-null  float64
 2   Torque_max   398844 non-null  float64
 3   Angle_min    398844 non-null  float64
 4   Angle_max    398844 non-null  float64
 5   Pset_name    398844 non-null  object 
 6   VIN          172013 non-null  object 
 7   Identifier1  172013 non-null  object 
 8   Identifier3  168406 non-null  object 
 9   Resultbin    398943 non-null  bool   
dtypes: bool(1), float64(5), object(4)
memory usage: 30.8+ MB


Vamos a echar un vistazo a los valores que vamos a quitar. porque se va a reducir mucho la muestra.

In [42]:

df_NANs=dfX[['Pset_name','Identifier1','Identifier3','Resultbin']]
df_NANs.loc[df_NANs['Identifier1'].isnull(),'value_is_NaN'] = 'Yes'
df_NANs.loc[df_NANs['Identifier1'].notnull(), 'value_is_NaN'] = 'No'
df_NANs = df_NANs[df_NANs["value_is_NaN"].str.contains("Yes",na=False)]
df_NANs[['Pset_name','Resultbin']].groupby(by='Pset_name').count().sort_values(by='Resultbin',ascending=False)



C:\Users\guill\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\pandas\core\indexing.py:1684: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = infer_fill_value(value)
C:\Users\guill\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,Resultbin
Pset_name,
Sliders 22Nm_4x65-4_ISC,19342
Backrest_40Nm,10887
Seat frame assy_35Nm_2x59-2_ISC,10709
Bearing inclination cap_22Nm_2x59-1_ISC,10022
Inner carrier_22Nm_2x59-3_ISC,9927
...,...
Cushion slides_WOOD_10Nm+-2,80
Embocar Profiles 30Nm ISC,77
Slides 22Nm TX40 ISC,6


Estamos perdiendo valores OK y NOK de las líneas NTS1 y NTS2 que no guardan valueIdentifiers. Es decir que el analisis se va a focalizar en las líneas:
1. ALter BAU
2. Alter BUS
3. Tapizado NTS1
4. Tapizado NTS2
5. NTS1 BAU
Vamos a proceder a eliminar los resultados nulos del Df

In [43]:
dfX=dfX.dropna()
dfX = dfX[pd.to_numeric(dfX['Identifier3'],errors='coerce').notna()]
dfX.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 168002 entries, 10 to 399996
Data columns (total 10 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   Step_ID      168002 non-null  float64
 1   Torque_min   168002 non-null  float64
 2   Torque_max   168002 non-null  float64
 3   Angle_min    168002 non-null  float64
 4   Angle_max    168002 non-null  float64
 5   Pset_name    168002 non-null  object 
 6   VIN          168002 non-null  object 
 7   Identifier1  168002 non-null  object 
 8   Identifier3  168002 non-null  object 
 9   Resultbin    168002 non-null  bool   
dtypes: bool(1), float64(5), object(4)
memory usage: 13.0+ MB


Observamos ahora que tenemos 168.002 registros no nulos con los que podemos elaborar un modelo. vamos a ver el porcentaje de aprietes NOK totales.  

In [44]:
dfX[['Resultbin','Step_ID']].groupby(by='Resultbin').count()

,Step_ID
Resultbin,
False,9236
True,158766


In [45]:
OKS = len(df[df["Result_status"].str.contains("OK",na=False)])
NOKS=len(df[df["Result_status"].str.contains("NOK",na=False)])
NOKS/OKS

0.05391998355654818

Tenemos un 5% de pares malos en este conjunto de datos.

In [46]:
enc=LabelEncoder()
dfX=df[['Torque_min','Torque_max','Angle_min','Angle_max','Pset_name',	'Identifier1','Identifier3','Resultbin']].dropna()
dfX['Identifier3'] = df['Identifier3'].astype('string',copy=False)
dfX['Pset_name_cat'] = enc.fit_transform(dfX['Pset_name'])
dfX['Modelo'] = enc.fit_transform(dfX['Identifier1'])
dfX[['Identifier3','Resultbin']].groupby(by="Identifier3").count()
dfX[dfX['Identifier3'].apply(lambda x: x.isnumeric())]
dfX['Trabajador'] = enc.fit_transform(dfX['Identifier3'])


In [47]:
dfX


,Torque_min,Torque_max,Angle_min,Angle_max,Pset_name,Identifier1,Identifier3,Resultbin,Pset_name_cat,Modelo,Trabajador
10,14.0,18.0,0.0,3000.0,Guide bolt 16Nm TX40 N,######814821-01/00,2356.0,True,70,115,38
12,30.0,40.0,0.0,3000.0,Belt 3rd point 35Nm TX45 ISC,######814821-01/00,2356.0,True,24,115,38
13,30.0,40.0,0.0,3000.0,Belt reel 35Nm TX45 ISC,######814821-01/00,2356.0,True,36,115,38
17,7.0,13.0,0.0,3000.0,Bearing bolt 10Nm TX30 N,######814821-01/00,2369.0,True,19,115,45
18,20.0,22.0,0.0,4000.0,Inner carrier 21Nm HEX8 ISC,552002110602000,2401.0,True,73,312,73
...,...,...,...,...,...,...,...,...,...,...,...
399990,20.0,30.0,0.0,500.0,Belt in backrest frame 25Nm TX45 ISC,######815202-02/00,2395.0,True,29,120,67
399991,30.0,40.0,0.0,3000.0,Seat frame assy 35Nm TX40 ISC,#######89161-05/00,2362.0,True,90,41,42
399994,30.0,40.0,0.0,350.0,Belt buckle 35Nm TX45 ISC,#######89161-05/00,2455.0,True,27,41,89
399995,20.0,24.0,0.0,220.0,Bearing inclination cap 22Nm HEX8 ISC,#######89161-05/00,2362.0,True,21,41,42


Ok, ahora vamos a dejar el df solo con las columnas que nos interesan y empezaremos a preparar nuestro modelo.

In [48]:
y=np.array(dfX['Resultbin'])
dfX=dfX.drop(columns={'Resultbin','Pset_name','Identifier1','Identifier3'})
X=dfX.values


In [49]:
from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy

scaler = MinMaxScaler(feature_range=(0,1))
scaled_samples= scaler.fit_transform(y.reshape(-1,1))



kf=KFold(n_splits=3, shuffle=True)